# Setup

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.nonparametric.kde import KDEUnivariate
from statsmodels.nonparametric import smoothers_lowess
from pandas import Series, DataFrame
from patsy import dmatrices
from sklearn import datasets, svm

In [2]:
from csv_utils import read_csv

# Data

In [3]:
#df  =pd.read_csv( "SPECTF.dat", header= None)
df = read_csv( "SPECTF.dat", header= None)    ##my read_csv

# Analyse

In [4]:
df.shape

(267, 45)

In [5]:
df.iloc[266,]

0      0
1     64
2     66
3     68
4     71
5     62
6     64
7     74
8     73
9     63
10    67
11    66
12    74
13    70
14    74
15    59
16    64
17    75
18    73
19    70
20    66
21    79
22    81
23    79
24    78
25    61
26    62
27    76
28    72
29    67
30    67
31    71
32    75
33    65
34    62
35    70
36    69
37    68
38    65
39    75
40    72
41    62
42    64
43    57
44    54
Name: 266, dtype: object

In [6]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,44
0,1,59,52,70,67,73,66,72,61,58,...,66,56,62,56,72,62,74,74,64,67
1,1,72,62,69,67,78,82,74,65,69,...,65,71,63,60,69,73,67,71,56,58
2,1,71,62,70,64,67,64,79,65,70,...,73,70,66,65,64,55,61,41,51,46
3,1,69,71,70,78,61,63,67,65,59,...,61,61,66,65,72,73,68,68,59,63
4,1,70,66,61,66,61,58,69,69,72,...,67,69,70,66,70,64,60,55,49,41


In [7]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,44
count,267,267,267,267,267,267,267,267,267,267,...,267,267,267,267,267,267,267,267,267,267
unique,2,39,43,34,40,44,44,37,42,37,...,36,44,37,40,51,50,56,61,52,59
top,1,68,66,70,74,67,69,76,72,68,...,66,66,70,69,73,67,74,68,56,58
freq,212,19,19,22,29,20,20,23,22,23,...,35,23,22,21,19,15,19,18,17,16


In [8]:
y = df[0]
x = df

In [9]:
x = x.drop([0], axis = 1)

In [10]:
x.head()

,1,2,3,4,5,6,7,8,9,10,...,35,36,37,38,39,40,41,42,43,44
0,59,52,70,67,73,66,72,61,58,52,...,66,56,62,56,72,62,74,74,64,67
1,72,62,69,67,78,82,74,65,69,63,...,65,71,63,60,69,73,67,71,56,58
2,71,62,70,64,67,64,79,65,70,69,...,73,70,66,65,64,55,61,41,51,46
3,69,71,70,78,61,63,67,65,59,59,...,61,61,66,65,72,73,68,68,59,63
4,70,66,61,66,61,58,69,69,72,68,...,67,69,70,66,70,64,60,55,49,41


In [11]:
np.array(y)

array(['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1

# Accuracy

In [12]:
def accuracy_score(Y_true, Y_predict):
    res = 0
    for i in range(len(Y_true)):
        if Y_true[i] == Y_predict[i]:
            res = res+ 1
    return res / len(Y_true)

# Evaluate Random Forest & Logistic Regression

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [14]:
trials = 30
all_accuracies_random_forest = np.zeros(trials)

all_accuracies_logistic_regression = np.zeros(trials)

### Table to fill

In [22]:
table = pd.DataFrame(np.nan, index=range(trials), columns=['Train Dataset size', 'Test Dataset Size', 'RandomForest tree','RandomForest max_depth','Accuracy RandomForest','Accuracy LogisticRegression'])

In [23]:
table

,Train Dataset size,Test Dataset Size,RandomForest tree,RandomForest max_depth,Accuracy RandomForest,Accuracy LogisticRegression
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
X= df
Y=df[0]

In [26]:
##different size of train and test 
size= list(range(37,95,3))
tree_number = 10
depth = 20

Evaluate the performance of Random Forest and Logistic Regression average over 20 trials of 4-fold cross validation

In [27]:
for trial in range(trials):    
    X= df
    Y=df[0]
    idx = np.arange(X.shape[0])
    np.random.seed(trial*500+12*2323)
    np.random.shuffle(idx)
    
    ##Everytime change the size of training and testing data:
    
    X = X.iloc[idx]
    Y = Y.iloc[idx]
    
    Xtrain = X[ 0: int(np.floor(size[int(trial%len(size))] * X.shape[0] / 100)) ]
    Ytrain = Y[ 0: int(np.floor(size[int(trial%len(size))] * X.shape[0] / 100)) ]

    Xtest = X[int(np.floor(size[int(trial%len(size))] * X.shape[0] / 100)): ]
    Ytest = Y[int(np.floor(size[int(trial%len(size))] * X.shape[0] / 100)): ]
    
    if(trial%5 ==0):
        tree_number = tree_number + 5
        depth = depth + 5
    
    ####RANDOM FOREST####
    
    # Create a random forest classifier. By convention, clf means 'classifier'
    clf = RandomForestClassifier(n_jobs=2, max_depth=depth, n_estimators=tree_number  )

    # Train the classifier to take the training features and learn how they relate
    # to the training y (the species)
    clf.fit(Xtrain, Ytrain)

    # output predictions on the remaining data
    Y_pred_random_forest = clf.predict(Xtest)
    all_accuracies_random_forest[trial]  = accuracy_score(np.array(Ytest), Y_pred_random_forest)
    ########
        
    ####Logistic Regression####
        
    clf2 = LogisticRegression()
    clf2.fit(Xtrain,Ytrain)

    # output predictions on the remaining data
    Y_pred_logistic = clf2.predict(Xtest)
        
    all_accuracies_logistic_regression[trial] = accuracy_score(np.array(Ytest.ravel()), np.array(Y_pred_logistic))
    ########
       
    
    print(trial, ":  ","Random Forest: ",all_accuracies_random_forest[trial],", Logistic Regression: ", all_accuracies_logistic_regression[trial])
    table.loc[trial, 'Train Dataset size'] = Xtrain.shape[0]
    table.loc[trial, 'Test Dataset Size'] = Xtest.shape[0]
    table.loc[trial, 'Accuracy RandomForest'] = all_accuracies_random_forest[trial]
    table.loc[trial, 'Accuracy LogisticRegression'] = all_accuracies_logistic_regression[trial]
    table.loc[trial, 'RandomForest tree'] = tree_number
    table.loc[trial, 'RandomForest max_depth'] = depth
    

0 :   Random Forest:  0.923076923077 , Logistic Regression:  0.715976331361
1 :   Random Forest:  0.944099378882 , Logistic Regression:  0.695652173913
2 :   Random Forest:  0.993464052288 , Logistic Regression:  0.81045751634
3 :   Random Forest:  0.993103448276 , Logistic Regression:  0.765517241379
4 :   Random Forest:  0.941605839416 , Logistic Regression:  0.802919708029
5 :   Random Forest:  1.0 , Logistic Regression:  0.860465116279
6 :   Random Forest:  0.97520661157 , Logistic Regression:  0.785123966942
7 :   Random Forest:  1.0 , Logistic Regression:  0.893805309735
8 :   Random Forest:  0.990476190476 , Logistic Regression:  0.904761904762
9 :   Random Forest:  1.0 , Logistic Regression:  0.948453608247
10 :   Random Forest:  0.910112359551 , Logistic Regression:  0.943820224719
11 :   Random Forest:  0.925925925926 , Logistic Regression:  0.938271604938
12 :   Random Forest:  0.972602739726 , Logistic Regression:  0.931506849315
13 :   Random Forest:  1.0 , Logistic Regres

In [28]:
table

,Train Dataset size,Test Dataset Size,RandomForest tree,RandomForest max_depth,Accuracy RandomForest,Accuracy LogisticRegression
0,98.0,169.0,15.0,25.0,0.923077,0.715976
1,106.0,161.0,15.0,25.0,0.944099,0.695652
2,114.0,153.0,15.0,25.0,0.993464,0.810458
3,122.0,145.0,15.0,25.0,0.993103,0.765517
4,130.0,137.0,15.0,25.0,0.941606,0.802920
5,138.0,129.0,20.0,30.0,1.000000,0.860465
6,146.0,121.0,20.0,30.0,0.975207,0.785124
7,154.0,113.0,20.0,30.0,1.000000,0.893805
8,162.0,105.0,20.0,30.0,0.990476,0.904762
9,170.0,97.0,20.0,30.0,1.000000,0.948454


We are getting accuracy almost 100% for Random Forest :D 

### with cross validation:

In [29]:
trials = 20
all_accuracies_random_forest = np.zeros(trials)

all_accuracies_logistic_regression = np.zeros(trials)

folds = 4  
fold_part = int(np.floor(X.shape[0] / folds))

In [30]:
for trial in range(trials):
    index_array = np.arange(X.shape[0])
    np.random.seed(trial*500 +23)
    np.random.shuffle(index_array)
    accuracy_random_forest = np.zeros(folds)
    accuracy_logistic_regression = np.zeros(folds)
    
        
    
    for i in range(folds):
        row_idx_train = index_array[np.r_[0: i * fold_part - 0, (i + 1) * fold_part: X.shape[0]]]
        Xtrain = X.iloc[row_idx_train[:, ]]
        Ytrain = Y.iloc[row_idx_train]

        row_idx_test = index_array[i * fold_part: (i + 1) * fold_part]
        Xtest = X.iloc[row_idx_test[:, ]]
        Ytest = Y.iloc[row_idx_test]

        ####RANDOM FOREST####
        
        # Create a random forest classifier. By convention, clf means 'classifier'
        clf = RandomForestClassifier(n_jobs=2, max_depth=20  )

        # Train the classifier to take the training features and learn how they relate
        # to the training y (the species)
        clf.fit(Xtrain, Ytrain)

        # output predictions on the remaining data
        Y_pred_random_forest = clf.predict(Xtest)
        accuracy_random_forest[i] = accuracy_score(np.array(Ytest), Y_pred_random_forest)
        ########
        
        ####Logistic Regression####
        
        clf2 = LogisticRegression()
        clf2.fit(Xtrain,Ytrain)

         # output predictions on the remaining data
        Y_pred_logistic = clf2.predict(Xtest)
        
        accuracy_logistic_regression[i] = accuracy_score(np.array(Ytest.ravel()), np.array(Y_pred_logistic))
        ########
       
    
    print(trial, ":  ","Random Forest: ",np.mean(accuracy_random_forest),", Logistic Regression: ", np.mean(accuracy_logistic_regression))
    table['Train Dataset size'] = Xtrain.shape[0]
    table['Test Dataset Size'] = Xtrain.shape[0]
    
    all_accuracies_random_forest[trial] = np.mean(accuracy_random_forest)
    all_accuracies_logistic_regression[trial] = np.mean(accuracy_logistic_regression)

0 :   Random Forest:  0.950757575758 , Logistic Regression:  0.939393939394
1 :   Random Forest:  0.977272727273 , Logistic Regression:  0.950757575758
2 :   Random Forest:  0.973484848485 , Logistic Regression:  0.92803030303
3 :   Random Forest:  0.962121212121 , Logistic Regression:  0.954545454545
4 :   Random Forest:  0.981060606061 , Logistic Regression:  0.954545454545
5 :   Random Forest:  0.969696969697 , Logistic Regression:  0.909090909091
6 :   Random Forest:  0.988636363636 , Logistic Regression:  0.924242424242
7 :   Random Forest:  0.988636363636 , Logistic Regression:  0.920454545455
8 :   Random Forest:  0.962121212121 , Logistic Regression:  0.935606060606
9 :   Random Forest:  0.981060606061 , Logistic Regression:  0.935606060606
10 :   Random Forest:  0.973484848485 , Logistic Regression:  0.916666666667
11 :   Random Forest:  0.981060606061 , Logistic Regression:  0.920454545455
12 :   Random Forest:  0.958333333333 , Logistic Regression:  0.935606060606
13 :   Ran

In [31]:
meanRandomForestAccuracy = np.mean(all_accuracies_random_forest)
stddevRandomForestAccuracy = np.std(all_accuracies_random_forest)

meanLogisticRegressionAccuracy = np.mean(all_accuracies_logistic_regression)
stddevLogisticRegressionAccuracy = np.std(all_accuracies_logistic_regression)

In [32]:
print("### Logistic Regression ###")
print("Accuracy: ", meanLogisticRegressionAccuracy)
print("Standard Deviation: ", stddevLogisticRegressionAccuracy)

### Logistic Regression ###
Accuracy:  0.938636363636
Standard Deviation:  0.0161862580449


#### Logistic Regression

Accuracy:  0.938636363636

Standard Deviation:  0.0161862580449

In [33]:
print("### Random Forest ###")
print("Accuracy: ", meanRandomForestAccuracy)
print("Standard Deviation: ", stddevRandomForestAccuracy)

### Random Forest ###
Accuracy:  0.975378787879
Standard Deviation:  0.0120678275167


#### Random Forest
Accuracy:  0.975378787879

Standard Deviation:  0.0120678275167

Wooow both are doing very very well !!

# More Examinations...

In [34]:
features = df.columns[1:df.shape[1]]
y = df[0]

In [35]:
# Create a random forest classifier. By convention, clf means 'classifier'
clf = RandomForestClassifier(n_jobs=2)

# Train the classifier to take the training features and learn how they relate
# to the training y (the species)
clf.fit(df[features], y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=2, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [36]:
predicted = clf.predict(df[features])

In [37]:
predicted[0:5]

array(['1', '1', '1', '1', '1'], dtype=object)

In [38]:
predicted_probs = clf.predict_proba(df[features])

In [39]:
predicted_probs[:10]

array([[ 0.1,  0.9],
       [ 0.1,  0.9],
       [ 0. ,  1. ],
       [ 0. ,  1. ],
       [ 0. ,  1. ],
       [ 0.1,  0.9],
       [ 0.1,  0.9],
       [ 0. ,  1. ],
       [ 0.1,  0.9],
       [ 0. ,  1. ]])

# Evaluate classifier

In [40]:
# Create confusion matrix
pd.crosstab(df[0], predicted, rownames=['Actual'], colnames=['Predicted'])

Predicted,0,1
Actual,,
0,53,2
1,1,211


In [41]:
print('accuracy: ', accuracy_score(y, predicted))

accuracy:  0.9887640449438202


In [42]:
# View a list of the features and their importance scores
list(zip(df[features], clf.feature_importances_))

[(1, 0.0070994559311820275),
 (2, 0.025781623070008676),
 (3, 0.029902746914809115),
 (4, 0.015635486117465191),
 (5, 0.0045115207373271887),
 (6, 0.027123256233772507),
 (7, 0.0091613322216075686),
 (8, 0.0090961213711069199),
 (9, 0.030217831100609972),
 (10, 0.022435677005519135),
 (11, 0.014998501331797212),
 (12, 0.023258568747499417),
 (13, 0.0),
 (14, 0.014967046594132824),
 (15, 0.038487393114161772),
 (16, 0.032747615715042644),
 (17, 0.020569400984547969),
 (18, 0.010481482194249136),
 (19, 0.011718545675223745),
 (20, 0.008383354597527367),
 (21, 0.0059843802466066822),
 (22, 0.0065041969358638884),
 (23, 0.013103696965678244),
 (24, 0.020425388457935949),
 (25, 0.028710214757173148),
 (26, 0.033334893768272783),
 (27, 0.022139103859055105),
 (28, 0.013852802412795313),
 (29, 0.014897856687581063),
 (30, 0.028478327955344024),
 (31, 0.02080712920360353),
 (32, 0.036219602127013119),
 (33, 0.066568534261240625),
 (34, 0.037104166780454265),
 (35, 0.015172608350232718),
 (36, 

we can take the features that have high imortance score, In this way we can reduce our computations and still get a good results

# Real Estate Data

In [43]:
dff = pd.read_csv("yerevan_april_9.csv") 

In [44]:
dff.head()

,Unnamed: 0,price,condition,district,max_floor,street,num_rooms,region,area,url,num_bathrooms,building_type,floor,ceiling_height
0,0,65000.0,good,Center,9,Vardanants St,3,Yerevan,80.0,http://www.myrealty.am/en/item/24032/3-senyaka...,1,panel,4,2.8
1,1,140000.0,newly repaired,Arabkir,10,Hr.Kochar St,4,Yerevan,115.0,http://www.myrealty.am/en/item/17599/4-senyaka...,1,monolit,2,3.0
2,2,97000.0,newly repaired,Center,10,Teryan St,2,Yerevan,72.0,http://www.myrealty.am/en/item/29472/2-senyaka...,1,panel,3,2.8
3,3,47000.0,good,Center,9,D. Demirchyan St,1,Yerevan,43.0,http://www.myrealty.am/en/item/37186/1-senyaka...,1,panel,9,2.8
4,4,51000.0,newly repaired,Center,14,Sayat Nova Ave,1,Yerevan,33.0,http://www.myrealty.am/en/item/37184/1-senyaka...,1,other,4,2.8


In [45]:
list(dff)

['Unnamed: 0',
 'price',
 'condition',
 'district',
 'max_floor',
 'street',
 'num_rooms',
 'region',
 'area',
 'url',
 'num_bathrooms',
 'building_type',
 'floor',
 'ceiling_height']

In [46]:
dff = dff.drop(['Unnamed: 0'], axis = 1)

In [47]:
list(dff)

['price',
 'condition',
 'district',
 'max_floor',
 'street',
 'num_rooms',
 'region',
 'area',
 'url',
 'num_bathrooms',
 'building_type',
 'floor',
 'ceiling_height']

In [48]:
dff = dff.drop(['url'], axis = 1)
dff = dff.drop(['region'], axis = 1)

In [49]:
dff.shape

(6234, 11)

In [50]:
dff.describe()

,price,max_floor,num_rooms,area,num_bathrooms,floor,ceiling_height
count,6234.000000,6234.000000,6234.000000,6234.000000,6234.000000,6234.000000,6234.000000
mean,86190.637151,8.760507,2.694899,81.607154,1.167148,5.190728,2.895797
std,52165.737360,4.168007,0.826298,24.809064,0.409621,3.412264,0.145520
min,18500.000000,1.000000,1.000000,27.000000,1.000000,0.000000,2.600000
25%,50000.000000,5.000000,2.000000,65.000000,1.000000,3.000000,2.800000
50%,70000.000000,9.000000,3.000000,80.000000,1.000000,4.000000,2.800000
75%,107000.000000,12.000000,3.000000,97.000000,1.000000,7.000000,3.000000
max,550000.000000,24.000000,6.000000,149.000000,4.000000,22.000000,3.800000


In [51]:
dff['price'].describe()

count      6234.000000
mean      86190.637151
std       52165.737360
min       18500.000000
25%       50000.000000
50%       70000.000000
75%      107000.000000
max      550000.000000
Name: price, dtype: float64

In [52]:
any(dff['price'].isnull())

False

In [53]:
for i in list(dff):
    print(i, "column: has NaN values:",any(dff[i].isnull()))

price column: has NaN values: False
condition column: has NaN values: False
district column: has NaN values: False
max_floor column: has NaN values: False
street column: has NaN values: False
num_rooms column: has NaN values: False
area column: has NaN values: False
num_bathrooms column: has NaN values: False
building_type column: has NaN values: False
floor column: has NaN values: False
ceiling_height column: has NaN values: False


we don't have null values in the data

# Data Graphically:

In [ ]:
# specifies the parameters of our graphs
fig = plt.figure(figsize=(18,6), dpi=1600) 
alpha=alpha_scatterplot = 0.2 
alpha_bar_chart = 0.55

dff.building_type.value_counts().plot(kind='bar', alpha=alpha_bar_chart, fontsize =20)

# puts a title on our graph
plt.title("Building Type", fontsize =22)    
plt.show()

In [ ]:
list(dff)

In [ ]:
dff.head()

# Random Forest

Doing it another way:

In [ ]:
# Create an acceptable formula:
formula_ml = 'price ~ C(condition) + C(district) + max_floor + C(street)  + num_rooms  +num_bathrooms +  C(building_type) + floor + ceiling_height' 

In [ ]:
# import the machine learning library that holds the randomforest
import sklearn.ensemble as ske

# Create the random forest model and fit the model to our training data
y, x = dmatrices(formula_ml, data=dff, return_type='dataframe')

# RandomForestClassifier expects a 1 demensional NumPy array, so we convert
y = np.asarray(y).ravel()
#instantiate and fit our model
results_rf = ske.RandomForestClassifier(n_estimators=100).fit(x, y)

# Score the results
score = results_rf.score(x, y)
print("Mean accuracy of Random Forest Predictions on the data was:\n {0}".format(score))

It should be done in linear regression, I am not sure how it is doing well with Random Forest :D 

# THE END